In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 10 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

26 December 2022 ; 14:19:57


In [3]:
features = ['DWS','DBPM','STL','BLK','DRB','PA/G','BLK%','DRtg/100','MP']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/dpoy/dpoy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,12,0,5.8,1.1,2.0,0.542,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,1,1
1,Bobby Portis,PF,27,MIL,33,8,26.0,5.7,11.5,0.500,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,1
2,Brook Lopez,C,34,MIL,32,32,30.6,5.6,10.9,0.511,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,0
3,George Hill,PG,36,MIL,29,0,19.8,1.6,3.8,0.427,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,1
4,Giannis Antetokounmpo,PF,28,MIL,28,28,33.3,11.2,20.9,0.536,...,2.52,113.63,111.03,2.6,2.73,113.8,110.96,2.84,0,0


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player' ,'Tm', 'DBPM', 'STL', 'BLK', 'DRtg/100', 'DWS', 'Seed', 'Share']]
res.head(5)

,Player,Tm,DBPM,STL,BLK,DRtg/100,DWS,Seed,Share
0,Nikola Jokić,DEN,3.9,1.5,0.7,110,3.675862,1,0.132317
1,Domantas Sabonis,SAC,2.3,0.8,0.5,111,3.703226,6,0.117514
2,Anthony Davis,LAL,2.0,1.3,2.1,108,4.592000,13,0.109534
3,Jrue Holiday,MIL,0.6,1.5,0.5,110,3.784615,2,0.096709
4,Jaren Jackson Jr.,MEM,2.8,1.0,3.1,102,4.612500,2,0.088277


In [7]:
res.to_csv(path + f'Results/dpoy/results_week_{week}.csv', index = None)